In [1]:
import numpy as np
import math

## projections

In [3]:
#projection onto a vector
def proj_b_vec(b,a):
#     proj=np.dot(b,a)/(np.dot(a,a))*a
    proj=(a.T@b)/(a.T@a)*a
    return proj

proj_b_vec(np.array([1,2,2]),np.array([1,1,1]))

array([1.66666667, 1.66666667, 1.66666667])

In [4]:
#Ax=b => A.T@Ax=A.T@b => x=(A.T@A)^(-1)@A.T@b => Ax=A@#Ax=b => A.T@Ax=A.T@b 
#x=(A.T@A)^(-1)@A.T@b => Ax=A@(A.T@A)^(-1)@A.T@b

def proj_b_plane(b,A):
    proj=A@(np.linalg.inv(A.T@A))@A.T@b
    return proj
proj_b_plane(np.array([1,1,0]),
             np.array([[1,0],[0,1],[1,1]]))


array([0.33333333, 0.33333333, 0.66666667])

In [9]:
q,r=np.linalg.qr(np.array([[1,-4],
                          [2,3],
                           [2,2]]))
print(q,"\n",r)

[[-0.33333333  0.93333333]
 [-0.66666667 -0.33333333]
 [-0.66666667 -0.13333333]] 
 [[-3. -2.]
 [ 0. -5.]]


# Chapter 6: Linear Least Squares Problems
solve Ax=b

Since there is no solution, we can try to minize $||b-Ax||_2$
### Normal equations
$A^T Ax=A^Tb$

### QR factorization: Gram-Schmidt algorithm
A matrix Q is orthogonal if its columns are orthonormal or if *Q.T@Q=I*.
### QR factorization: Householder reflectors

## Normal equations

### Library used
1. np.linalg.lstsq: https://numpy.org/doc/stable/reference/generated/numpy.linalg.lstsq.html
2. np.linalg.cholesky: https://numpy.org/doc/stable/reference/generated/numpy.linalg.cholesky.html
* cholesky returns the lower-triangular Cholesky factor of a
3. np.linalg.solve:https://numpy.org/doc/stable/reference/generated/numpy.linalg.solve.html

### Pros and cons
1. fast and straightforward
2. not as stable as it can be in general: if you have large condition numbers


In [2]:
#using np.linalg.lstsq
def normal_equations1(A,b):
    x=np.linalg.lstsq(A,b,rcond=None)
    print("Estimate of x:",x[0])
    
    r=b-A@x[0]#compute residual, x[0] is our estimate of x to Ax=b
    print("Residual r=b-Ax:",r)
    #Are r and A.T perpendicular? They should be perpendicular
    print("L2 norm of A.T@r:",np.linalg.norm(A.T@r))
    print("Residual L2 norm:",math.sqrt(x[1]))#np.linalg.lstsq returns sums of squared residuals
    

A=np.array([[1,0,1],
           [2,3,5],
           [5,3,-2],
           [3,5,4],
           [-1,6,3]])
b=np.array([4,-2,5,-2,1])
normal_equations1(A,b)

Estimate of x: [ 0.34722617  0.39900427 -0.7859175 ]
Residual r=b-Ax: [ 4.43869132  0.03812233  0.49502134 -1.89302987  1.31095306]
L2 norm of A.T@r: 1.890380887718447e-14
Residual L2 norm: 5.025001503860272


In [16]:
data=np.array([[1,1],
               [1.1,1],
               [1.3,1],
              [1.5,1],
               [1.9,1],
               [2.1,1]
              ])
y=np.array([1.84,1.96,2.21,2.45,2.94,3.18])
normal_equations1(data,y)

Estimate of x: [1.21962134 0.62089501]
Residual r=b-Ax: [-0.00051635 -0.00247849  0.00359725 -0.00032702  0.00182444 -0.00209983]
L2 norm of A.T@r: 8.022137326523255e-15
Residual L2 norm: 0.005214833866456243


In [17]:
normal_equations1(np.array([[1,1],
                            [1,0],
                            [0,1]]),
                  np.array([-6,-6,-6]))

Estimate of x: [-4. -4.]
Residual r=b-Ax: [ 2. -2. -2.]
L2 norm of A.T@r: 7.640399661428378e-15
Residual L2 norm: 3.464101615137754


In [18]:
normal_equations1(np.array([[6,9],
                            [3,8],
                            [2,10]]),
                  np.array([49,0,0]))

Estimate of x: [12. -3.]
Residual r=b-Ax: [  4. -12.   6.]
L2 norm of A.T@r: 3.243687902600336e-13
Residual L2 norm: 14.000000000000004


In [11]:
math.sqrt(4**2+12**2+6**2)
np.linalg.norm(np.array([4,-12,6]))

14.0

In [8]:
np.array([[6,9],[3,8],[2,10]])@np.array([12,-3])

array([45, 12, -6])

In [3]:
#using the definition A.T Ax=A.T b

def normal_eqn2(A,b):
    B=A.T@A
    y=A.T@b
    
    #B is symmetric, pos def, where Cholesky factorization comes in
    G=np.linalg.cholesky(B)
    
    #Bx=y => G G.T x=y; using forward substitution and backward substitution
    z=np.linalg.solve(G,y)
    x=np.linalg.solve(G.T,z)
    print(x)
    r=b-A@x
    print("Residual r=b-A@x:",r)
    print("A.T @ r:",A.T@r)
normal_eqn2(A,b)

[ 0.34722617  0.39900427 -0.7859175 ]
Residual r=b-A@x: [ 4.43869132  0.03812233  0.49502134 -1.89302987  1.31095306]
A.T @ r: [ 1.77635684e-15  0.00000000e+00 -4.44089210e-15]


## QR factorization
more computational expensive than normal equations but is more robust
1. steps
* Ax=b
* QRx=b
* (Q.T@Q)@Rx=Q.T@b
* Rx=Q.T@b
2. libraries
np.linalg.qr: https://numpy.org/doc/stable/reference/generated/numpy.linalg.qr.html

In [19]:
def QR_fac(A,b):
    Q,R=np.linalg.qr(A)
    x=np.linalg.solve(R,Q.T@b)
    print(x)
QR_fac(A,b)

[ 0.34722617  0.39900427 -0.7859175 ]


In [4]:
def gram_schmidt(X):
    O = np.zeros(X.shape)
    for i in range(X.shape[1]):
        # orthogonalization
        vector = X[:, i]
        space = O[:, :i]
        projection = vector @ space
        vector = vector - np.sum(projection * space, axis=1)
        # normalization
        norm = np.sqrt(vector @ vector)
        vector /= abs(norm) < 1e-8 and 1 or norm
        
        O[:, i] = vector
    return O

gram_schmidt(np.array([[1,-4],
                          [2,3],
                           [2,2]]))

array([[ 0.33333333, -0.93333333],
       [ 0.66666667,  0.33333333],
       [ 0.66666667,  0.13333333]])

In [12]:
def QR_Decomposition(A):
    n, m = A.shape # get the shape of A

    Q = np.empty((n, n)) # initialize matrix Q
    u = np.empty((n, n)) # initialize matrix u

    u[:, 0] = A[:, 0]
    Q[:, 0] = u[:, 0] / np.linalg.norm(u[:, 0])

    for i in range(1, n):

        u[:, i] = A[:, i]
        for j in range(i):
            u[:, i] -= (A[:, i] @ Q[:, j]) * Q[:, j] # get each u vector

        Q[:, i] = u[:, i] / np.linalg.norm(u[:, i]) # compute each e vetor

    R = np.zeros((n, m))
    for i in range(n):
        for j in range(i, m):
            R[i, j] = A[:, j] @ Q[:, i]

    return Q, R

In [14]:
QR_Decomposition(np.array([[1,1,0],
                          [1,2,-3],
                           [1,3,9.1]]))

(array([[ 5.77350269e-01, -7.07106781e-01,  4.08248290e-01],
        [ 5.77350269e-01, -3.14018492e-16, -8.16496581e-01],
        [ 5.77350269e-01,  7.07106781e-01,  4.08248290e-01]]),
 array([[1.73205081, 3.46410162, 3.52183664],
        [0.        , 1.41421356, 6.43467171],
        [0.        , 0.        , 6.16454919]]))

In [19]:
q,r=np.linalg.qr(np.array([[1,1,0],
                          [1,2,-3],
                           [1,3,9.1]]))
q@r

array([[ 1. ,  1. ,  0. ],
       [ 1. ,  2. , -3. ],
       [ 1. ,  3. ,  9.1]])